In [ ]:
import polars
import matplotlib.pyplot as plt
import torchaudio

import helpers.input_processor as ip

In [ ]:
data_dir = "data/raw_training/training_data/"
df = ip.ingest_data(data_dir)

In [ ]:
# output metadata of first audio file
metadata = torchaudio.info(data_dir + df['audio_file'][0][0])
print(metadata)